# CIFAR-10 Classification with CNN

In [1]:
import torch
import torchvision
from torchvision import datasets

from torch import nn
from torch import optim

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

import math
import numpy as np
import matplotlib.pyplot as plt

import neural_nets
import cnn_utils as util

In [2]:
torch.manual_seed(42)

## Import Data

### Download Dataset

In [3]:
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Files already downloaded and verified
Files already downloaded and verified


### Create Loaders

This step is needed in order to automate the loading of images with set batch size.

In [4]:
batch_size = 256

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=batch_size,
                                          shuffle=True)

### Set general parameters

In [6]:
input_dimensions = (32,32)
target_classes = 10

# Model Optimization

### Training hyperparameters

In [7]:
num_epochs = 30
learning_rate = 0.001

## 2: Layer number

The sets of parameters that are tested here will be chosen according to the results of the previous experiments.

### 3 Layers

In [8]:
filter_params = [[32, 64, 128], [64, 128, 256]]
kernel_params = [[3, 3, 3], [5, 5, 5]]
fulcon_params = [[64], [128], [256]]

#### Training

In [9]:
for f in filter_params:
    for k in kernel_params:
        for fci in range(len(fulcon_params)):
            fc = fulcon_params[fci].copy()
            cnn = neural_nets.CNN(n_conv_layers=3,
                                  filters=f,
                                  kernel=k,
                                  activation=['relu', 'relu', 'relu'],
                                  norm=[False, False, False],
                                  pool=[2, 2, 2],
                                  input_channels=3,
                                  fully_connected=fc,
                                  input_dims=input_dimensions,
                                  classes=target_classes)
            cross_entropy = nn.CrossEntropyLoss()
            adam = optim.Adam(cnn.parameters(), lr=learning_rate)
            %time losses, accuracies = util.train_loop(cnn, cross_entropy, adam, train_loader, test_loader, num_epochs, 0)
            
            print("Max accuracy:", max(accuracies), "Epoch:", np.argmax(accuracies)+1, "Params:", f, k, fc)
            print(76*"-")

CPU times: user 1h 4min 35s, sys: 8min 5s, total: 1h 12min 40s
Wall time: 19min 19s
Max accuracy: 0.7490000128746033 Epoch: 19 Params: [32, 64, 128] [3, 3, 3] [2048, 64, 10]
----------------------------------------------------------------------------
CPU times: user 1h 3min 28s, sys: 8min 30s, total: 1h 11min 58s
Wall time: 19min 1s
Max accuracy: 0.7486000061035156 Epoch: 16 Params: [32, 64, 128] [3, 3, 3] [2048, 128, 10]
----------------------------------------------------------------------------
CPU times: user 1h 9min 2s, sys: 8min 42s, total: 1h 17min 44s
Wall time: 20min 29s
Max accuracy: 0.7372999787330627 Epoch: 19 Params: [32, 64, 128] [3, 3, 3] [2048, 256, 10]
----------------------------------------------------------------------------
CPU times: user 2h 37min 3s, sys: 16min 18s, total: 2h 53min 22s
Wall time: 24min 15s
Max accuracy: 0.7361000180244446 Epoch: 11 Params: [32, 64, 128] [5, 5, 5] [2048, 64, 10]
---------------------------------------------------------------------

### 4 Layers

In [13]:
filter_params = [[16, 32, 64, 128], [32, 64, 128, 256], [64, 128, 256, 512]]
kernel_params = [[3, 3, 3, 3]] # image is not big enough after 3 pooling layers to use larger kernels
fulcon_params = [[32], [64], [128], [256]]

#### Training

In [14]:
for f in filter_params:
    for k in kernel_params:
        for fci in range(len(fulcon_params)):
            fc = fulcon_params[fci].copy()
            cnn = neural_nets.CNN(n_conv_layers=4,
                                  filters=f,
                                  kernel=k,
                                  activation=['relu', 'relu', 'relu', 'relu'],
                                  norm=[False, False, False, False],
                                  pool=[2, 2, 2, 2],
                                  input_channels=3,
                                  fully_connected=fc,
                                  input_dims=input_dimensions,
                                  classes=target_classes)
            cross_entropy = nn.CrossEntropyLoss()
            adam = optim.Adam(cnn.parameters(), lr=learning_rate)
            %time losses, accuracies = util.train_loop(cnn, cross_entropy, adam, train_loader, test_loader, num_epochs, 0)
            
            print("Max accuracy:", max(accuracies), "Epoch:", np.argmax(accuracies)+1, "Params:", f, k, fc)
            print(76*"-")

CPU times: user 52min 54s, sys: 4min 42s, total: 57min 37s
Wall time: 10min 11s
Max accuracy: 0.7208999991416931 Epoch: 24 Params: [16, 32, 64, 128] [3, 3, 3, 3] [512, 32, 10]
----------------------------------------------------------------------------
CPU times: user 53min 41s, sys: 4min 40s, total: 58min 21s
Wall time: 9min 51s
Max accuracy: 0.7020999789237976 Epoch: 19 Params: [16, 32, 64, 128] [3, 3, 3, 3] [512, 64, 10]
----------------------------------------------------------------------------
CPU times: user 56min 39s, sys: 4min 42s, total: 1h 1min 21s
Wall time: 10min 6s
Max accuracy: 0.7168999910354614 Epoch: 15 Params: [16, 32, 64, 128] [3, 3, 3, 3] [512, 128, 10]
----------------------------------------------------------------------------
CPU times: user 59min 29s, sys: 4min 49s, total: 1h 4min 18s
Wall time: 10min 36s
Max accuracy: 0.7095999717712402 Epoch: 13 Params: [16, 32, 64, 128] [3, 3, 3, 3] [512, 256, 10]
-------------------------------------------------------------